In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bbc-news/bbc_news.csv


In [2]:
df = pd.read_csv("/kaggle/input/bbc-news/bbc_news.csv")
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [3]:
#EDA
df.shape

(42115, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42115 entries, 0 to 42114
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        42115 non-null  object
 1   pubDate      42115 non-null  object
 2   guid         42115 non-null  object
 3   link         42115 non-null  object
 4   description  42115 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


In [5]:
df.isnull().sum()

title          0
pubDate        0
guid           0
link           0
description    0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.sample(4)


,title,pubDate,guid,link,description
29211,Abdul Ezedi: Thames search for Clapham attack ...,"Sat, 10 Feb 2024 15:09:18 GMT",https://www.bbc.co.uk/news/uk-68260335,https://www.bbc.co.uk/news/uk-68260335?at_medi...,Detectives believe Abdul Shokoor Ezedi may hav...
37826,Presenter Woods received death threats after O...,"Thu, 15 Aug 2024 16:05:14 GMT",https://www.bbc.com/sport/olympics/articles/c5...,https://www.bbc.com/sport/olympics/articles/c5...,"Laura Woods says she received ""numerous death ..."
8725,Calais migrants: Smugglers cause fresh problem...,"Tue, 04 Oct 2022 05:00:05 GMT",https://www.bbc.co.uk/news/world-europe-63119180,https://www.bbc.co.uk/news/world-europe-631191...,New Calais smuggling networks are making it ev...
39442,Men and women starting salaries equal from 2025,"Tue, 24 Sep 2024 21:31:24 GMT",https://www.bbc.com/sport/cricket/articles/c0l...,https://www.bbc.com/sport/cricket/articles/c0l...,Minimum starting salaries will be equal for me...


In [8]:
# since we are doing topic modeling we dont need pubDate, guid and link 
df = df[['title','description']]
df.head()

,title,description
0,Ukraine: Angry Zelensky vows to punish Russian...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,Consumers are feeling the impact of higher ene...


In [9]:
#Combining these two columns
df['text'] = df['title'].astype(str) + " " + df['description'].astype(str)
df = df[['text']]   
df.head()



,text
0,Ukraine: Angry Zelensky vows to punish Russian...
1,War in Ukraine: Taking cover in a town under a...
2,Ukraine war 'catastrophic for global food' One...
3,Manchester Arena bombing: Saffie Roussos's par...
4,Ukraine conflict: Oil price soars to highest l...


In [10]:
#Preprocessing
import nltk
import spacy
import re
from nltk.corpus import stopwords


In [11]:
import en_core_web_sm

stop_words = set(stopwords.words('english'))
nlp = en_core_web_sm.load()

In [12]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc 
              if token.lemma_ not in stop_words and len(token) > 2]
    
    return tokens

In [13]:
df['tokens'] = df['text'].apply(preprocess)


In [14]:
df[['text', 'tokens']].head(10)


,text,tokens
0,Ukraine: Angry Zelensky vows to punish Russian...,"[ukraine, angry, zelensky, vow, punish, russia..."
1,War in Ukraine: Taking cover in a town under a...,"[war, ukraine, take, cover, town, attack, jere..."
2,Ukraine war 'catastrophic for global food' One...,"[ukraine, war, catastrophic, global, food, one..."
3,Manchester Arena bombing: Saffie Roussos's par...,"[manchester, arena, bombing, saffie, roussoss,..."
4,Ukraine conflict: Oil price soars to highest l...,"[ukraine, conflict, oil, price, soar, high, le..."
5,Ukraine war: PM to hold talks with world leade...,"[ukraine, war, hold, talk, world, leader, sanc..."
6,Ukraine war: UK grants 50 Ukrainian refugee vi...,"[ukraine, war, grant, ukrainian, refugee, visa..."
7,TikTok limits services as Netflix pulls out of...,"[tiktok, limit, service, netflix, pull, russia..."
8,"Covid: Fourth jab for Scotland's vulnerable, a...","[covid, fourth, jab, scotland, vulnerable, tes..."
9,Protests across Russia see thousands detained ...,"[protest, across, russia, see, thousand, detai..."


In [15]:
from gensim import corpora
texts = df['tokens'].tolist()
id2word = corpora.Dictionary(texts)
id2word.filter_extremes(no_below = 5, no_above =0.5)
corpus = [id2word.doc2bow(text) for text in texts]
print("Vocabulary Size: ", len(id2word))
print("Example doc below (first doc): ", corpus[0][:20])

Vocabulary Size:  10199
Example doc below (first doc):  [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]


In [16]:
#LDA MOdel Training
from gensim.models import LdaModel

lda = LdaModel(
    corpus = corpus,
    id2word = id2word,
    num_topics = 10,
    random_state = 42,
    chunksize = 2000,
    passes=10,
    alpha = 'auto',
    eta = 'auto'
)
for i in range(lda.num_topics):
    print(f"Topic {i:02d}:", ", ".join([w for w, p in lda.show_topic(i, topn=12)]))

Topic 00: die, man, police, woman, find, two, take, show, say, death, life, family
Topic 01: day, gaza, lead, paper, england, test, tory, past, sunak, medal, row, series
Topic 02: president, war, ukraine, attack, israeli, school, russian, strike, russia, force, biden, military
Topic 03: say, year, new, people, get, could, make, trump, bbc, first, face, rise
Topic 04: minister, leader, labour, budget, call, strike, deal, government, prime, talk, sir, nhs
Topic 05: collapse, prison, pitch, term, birth, sentence, letby, sea, suspend, holiday, chant, syria
Topic 06: win, world, england, cup, final, first, beat, second, open, champion, watch, victory
Topic 07: israel, wale, euro, watch, six, paris, ireland, moment, britain, medium, history, social
Topic 08: city, manchester, premier, league, liverpool, united, club, man, manager, football, tour, good
Topic 09: election, party, south, king, general, storm, award, flood, vote, result, journey, chef


In [17]:
lda.print_topics(num_words=10)


[(0,
  '0.019*"die" + 0.017*"man" + 0.016*"police" + 0.015*"woman" + 0.015*"find" + 0.013*"two" + 0.013*"take" + 0.012*"show" + 0.012*"say" + 0.011*"death"'),
 (1,
  '0.040*"day" + 0.030*"gaza" + 0.023*"lead" + 0.019*"paper" + 0.017*"england" + 0.017*"test" + 0.014*"tory" + 0.013*"past" + 0.012*"sunak" + 0.011*"medal"'),
 (2,
  '0.024*"president" + 0.023*"war" + 0.023*"ukraine" + 0.017*"attack" + 0.015*"israeli" + 0.015*"school" + 0.014*"russian" + 0.013*"strike" + 0.013*"russia" + 0.013*"force"'),
 (3,
  '0.047*"say" + 0.018*"year" + 0.016*"new" + 0.010*"people" + 0.009*"get" + 0.009*"could" + 0.009*"make" + 0.008*"trump" + 0.008*"bbc" + 0.008*"first"'),
 (4,
  '0.028*"minister" + 0.026*"leader" + 0.026*"labour" + 0.017*"budget" + 0.015*"call" + 0.015*"strike" + 0.013*"deal" + 0.013*"government" + 0.013*"prime" + 0.013*"talk"'),
 (5,
  '0.028*"collapse" + 0.016*"prison" + 0.015*"pitch" + 0.013*"term" + 0.012*"birth" + 0.012*"sentence" + 0.011*"letby" + 0.011*"sea" + 0.010*"suspend" + 

In [18]:
for idx, doc in enumerate(corpus[:3]):   
    print(f"Document {idx}")
    print(lda[doc])  
    print("\n")

Document 0
[(0, 0.10932584), (1, 0.022335974), (2, 0.61729795), (3, 0.11450058), (4, 0.026077848), (5, 0.013932211), (6, 0.02934993), (7, 0.026740141), (8, 0.025140794), (9, 0.015298706)]


Document 1
[(0, 0.15601344), (1, 0.024305172), (2, 0.5245193), (3, 0.15614167), (4, 0.028993245), (5, 0.013398808), (6, 0.030870674), (7, 0.026375914), (8, 0.02472796), (9, 0.014653741)]


Document 2
[(0, 0.059461255), (1, 0.020437256), (2, 0.27268514), (3, 0.43691456), (4, 0.022218209), (5, 0.012788069), (6, 0.0713548), (7, 0.06689948), (8, 0.023197059), (9, 0.014044178)]




In [19]:
#Evaluation
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda, 
                                     texts=df['tokens'],  
                                     dictionary=id2word, 
                                     coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_lda:.4f}")

Coherence Score: 0.4172


In [20]:
#Visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.219990 -0.077741       1        1  25.798587
0     -0.204778 -0.089650       2        1  17.443741
6     -0.098891  0.245178       3        1  11.971016
8     -0.067558  0.221357       4        1   8.342238
2     -0.086693 -0.229205       5        1   7.720897
7      0.071738  0.155565       6        1   7.526946
4      0.041999 -0.187392       7        1   6.854679
1      0.062628  0.052987       8        1   6.448513
9      0.227350 -0.047812       9        1   4.123590
5      0.274196 -0.043286      10        1   3.769794, topic_info=          Term          Freq         Total Category  logprob  loglift
467        win   3624.000000   3624.000000  Default  30.0000  30.0000
10         say  10298.000000  10298.000000  Default  29.0000  29.0000
962   election   1873.000000   1873.000000  Default  28.0000  28.0000
41       world   2752.000000   2752.000000  Default  27.0000  27.0000
440    england   3413.000000   3413.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
818       andy    191.940519    192.872791  Topic10  -4.9078   3.2733
2262   airport    182.655884    183.588153  Topic10  -4.9574   3.2731
1080     plane    179.659913    180.592181  Topic10  -4.9739   3.2730
341     answer    174.258393    175.190720  Topic10  -5.0044   3.2728
1069    appeal    173.192634    174.124960  Topic10  -5.0105   3.2728

[458 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2279      7  0.996479     able
672       2  0.998594   accuse
1236      7  0.998989   action
2070      9  0.996492   africa
3019      2  0.998233      age
...     ...       ...      ...
312       4  0.000898     year
312       9  0.000898     year
168       2  0.998973  yearold
2487      7  0.996149      yet
53        2  0.997964    young

[625 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 7, 9, 3, 8, 5, 2, 10, 6])

In [21]:
topics = lda.print_topics(num_words=10)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.019*"die" + 0.017*"man" + 0.016*"police" + 0.015*"woman" + 0.015*"find" + 0.013*"two" + 0.013*"take" + 0.012*"show" + 0.012*"say" + 0.011*"death"
Topic 1: 0.040*"day" + 0.030*"gaza" + 0.023*"lead" + 0.019*"paper" + 0.017*"england" + 0.017*"test" + 0.014*"tory" + 0.013*"past" + 0.012*"sunak" + 0.011*"medal"
Topic 2: 0.024*"president" + 0.023*"war" + 0.023*"ukraine" + 0.017*"attack" + 0.015*"israeli" + 0.015*"school" + 0.014*"russian" + 0.013*"strike" + 0.013*"russia" + 0.013*"force"
Topic 3: 0.047*"say" + 0.018*"year" + 0.016*"new" + 0.010*"people" + 0.009*"get" + 0.009*"could" + 0.009*"make" + 0.008*"trump" + 0.008*"bbc" + 0.008*"first"
Topic 4: 0.028*"minister" + 0.026*"leader" + 0.026*"labour" + 0.017*"budget" + 0.015*"call" + 0.015*"strike" + 0.013*"deal" + 0.013*"government" + 0.013*"prime" + 0.013*"talk"
Topic 5: 0.028*"collapse" + 0.016*"prison" + 0.015*"pitch" + 0.013*"term" + 0.012*"birth" + 0.012*"sentence" + 0.011*"letby" + 0.011*"sea" + 0.010*"suspend" + 0.010*"ho

it shows what my model has learned we can see in topic 0 it leans die, man , police, woman etc and so on 10 topics. sicne it is unigram so only 1 word

In [22]:
#Compare with NMF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = vectorizer.fit_transform(df['text'])  

nmf = NMF(n_components=5, random_state=42)
nmf_topics = nmf.fit_transform(tfidf)

feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")


Topic 0: world, cup, england, final, women, 2022, win, australia, wales, euro
Topic 1: uk, says, election, sunak, new, papers, minister, labour, government, rishi
Topic 2: ukraine, war, russia, russian, putin, invasion, ukrainian, kyiv, says, president
Topic 3: league, manchester, city, premier, united, liverpool, champions, win, man, arsenal
Topic 4: israel, police, gaza, year, hamas, people, killed, old, say, attack


now we can see the difference using NMF it extract words like word, cup , england but using LDA it extract words like die, man, police, women this shows each topic is tight and interpretable, with clear themes like crime, war, politics, sports, Israel/Gaza, etc.in LDA while in NMF it more focus on politics and sports etc.